# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si; psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.232025800420                   -0.50    7.0         
  2   -7.249288331021       -1.76       -1.39    1.0   5.14ms
  3   -7.251144691282       -2.73       -1.90    2.0   6.14ms
  4   -7.251081364345   +   -4.20       -1.95    2.0   6.25ms
  5   -7.251304383384       -3.65       -2.40    1.0   5.16ms
  6   -7.251337661150       -4.48       -3.03    1.0   5.22ms
  7   -7.251338712061       -5.98       -3.64    1.0   5.29ms
  8   -7.251338788636       -7.12       -4.03    2.0   6.43ms
  9   -7.251338797952       -8.03       -4.71    1.0   5.47ms
 10   -7.251338798621       -9.17       -5.15    2.0   6.52ms
 11   -7.251338798698      -10.11       -5.60    2.0   6.91ms
 12   -7.251338798704      -11.30       -6.07    1.0   5.77ms
 13   -7.251338798704      -12.04       -6.62    2.0   6.78ms
 14   -7.251338798705      -13.01       -7.01    2.0   6.79ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.047939192708114156
[ Info: Arnoldi iteration step 2: normres = 0.32081128364218453
[ Info: Arnoldi iteration step 3: normres = 0.7943953458977798
[ Info: Arnoldi iteration step 4: normres = 0.531913109205625
[ Info: Arnoldi iteration step 5: normres = 0.6003914819989754
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.68e-01, 7.88e-02, 5.31e-01, 2.09e-01, 2.92e-02)
[ Info: Arnoldi iteration step 6: normres = 0.2154351618007409
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.89e-02, 8.49e-02, 1.47e-01, 1.12e-01, 1.35e-02)
[ Info: Arnoldi iteration step 7: normres = 0.12403813082175659
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.71e-03, 9.50e-03, 1.62e-02, 7.88e-02, 9.13e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10691512021001262
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (7.96e-